In [7]:
import numpy as np
import glob
import sys
import os


In [8]:
from crispy.tools.initLogger import getLogger
log = getLogger('crispy')

In [3]:
os.chdir('/Users/mrizzo/IFS/crispy/crispy/WFIRST660/')
from params import Params
par = Params()
par.hdr

SIMPLE  =                    T / conforms to FITS standard                      
BITPIX  =                    8 / array data type                                
NAXIS   =                    0 / number of array dimensions                     
EXTEND  =                    T                                                  
COMMENT                                                                         
COMMENT ************************************************************            
COMMENT ********************** General parameters ******************            
COMMENT ************************************************************            
COMMENT                                                                         
NLENS   =                  108 / # lenslets across array                        
PITCH   =             0.000174 / Lenslet pitch (meters)                         
INTERLAC=                  2.0 / Interlacing                                    
PHILENS =    26.565051177077

# Create input cube

In [4]:
from crispy.tools.imgtools import gausspsf
from astropy.io import fits
lamc=660.
BW=0.4
lamlist = np.arange(lamc*(1.-BW/2.),lamc*(1.+BW/2.)+5.,5)

inputCube = np.ones((len(lamlist), 512, 512), dtype=float)
inCube = fits.HDUList(fits.PrimaryHDU(inputCube))
inCube[0].header['LAM_C'] = 0.66 # in microns
inCube[0].header['PIXSIZE'] = 0.1 # unit is lambda/D @lam_c
for i,lam in enumerate(lamlist):
    # replace this by your spatio-spectral scene of choice
    inCube[0].data[i] = gausspsf(size=512,fwhm=10*lam/lamc,offx=0.,offy=0.)

In [5]:
inCube.writeto('/Users/mrizzo/Downloads/testinputcube.fits',overwrite=True)

# Propagate

In [6]:
from crispy.IFS import polychromeIFS
dlam = lamlist[1]-lamlist[0]
par.saveRotatedInput=True
detectorFrame = polychromeIFS(
                            par,
                            lamlist,
                            inCube[0],
                            dlambda=dlam,
                            parallel=True,
                            lam_arr=lamlist)


crispy - INFO - The number of input pixels per lenslet is 5.000000
crispy - INFO - Using PSFlet gaussian approximation
crispy - WARNING - Assuming slices are evenly spread in wavelengths
crispy - INFO - Creating Gaussian PSFLet templates
crispy - INFO - Writing data to ..//SimResults/imagePlaneRot.fits
crispy - INFO - Writing data to ..//SimResults/detectorFramepoly.fits
crispy - INFO - Writing data to ..//SimResults/detectorFrame.fits
crispy - INFO - Done.
crispy - INFO - Performance: 74 seconds total
